In [103]:
import csv
import pandas as pd
from pandas import *
from __future__ import division
from pandas.tseries.offsets import *
import dateutil.rrule as dr
import dateutil.parser as dp
import dateutil.relativedelta as drel

## Loading files

In [88]:
spd = read_csv('spd.csv', delimiter=',', header=0)
spw = read_csv('spw.csv', delimiter=',', header=0)
options = read_csv('options.csv', delimiter=',', header=0)

In [89]:
spd.rename(columns={'Unnamed: 0':'nrow'}, inplace=True)
spw.rename(columns={'Unnamed: 0':'nrow'}, inplace=True)
options.rename(columns={'Unnamed: 0':'nrow'}, inplace=True)

In [90]:
spd['date'] = pd.to_datetime(spd['date'])
spw['date'] = pd.to_datetime(spw['date'])
print type(spd.date[0]), type(spw.date[0])

<class 'pandas.tslib.Timestamp'> <class 'pandas.tslib.Timestamp'>


In [91]:
spd.head(3)

,nrow,date,o,h,l,close,vol,c
0,0,2015-06-05,2095.09009,2100.98999,2085.66992,2092.83008,3243690000,2092.83008
1,1,2015-06-04,2112.35010,2112.88989,2093.22998,2095.84009,3200050000,2095.84009
2,2,2015-06-03,2110.63989,2121.91992,2109.61011,2114.07007,3099980000,2114.07007


In [92]:
spw.head(3)

,nrow,date,o,h,l,close,vol,c
0,0,2015-06-01,2108.63989,2121.91992,2085.66992,2092.83008,3120956000,2092.83008
1,1,2015-05-26,2125.34009,2126.21997,2099.17993,2107.38989,3344457500,2107.38989
2,2,2015-05-18,2121.30005,2134.71997,2120.01001,2126.06006,2970484000,2126.06006


In [134]:
len(spd)

10452

In [133]:
len(spw)

2161

In [93]:
#rolling_mean(spd.close, window = 288, min_periods=1, center=False)

## Moving Averages

In [95]:
d12 = []
d24 = []
d48 = []
d72 = []
d96 = []
d192 = []
d288 = []
for i in range(len(spd)-287):
    d12.append(sum(spd.c[i:(i+12)])/12)
    d24.append(sum(spd.c[i:(i+24)])/24)
    d48.append(sum(spd.c[i:(i+48)])/48)
    d72.append(sum(spd.c[i:(i+72)])/72)
    d96.append(sum(spd.c[i:(i+96)])/96)
    d192.append(sum(spd.c[i:(i+192)])/192)
    d288.append(sum(spd.c[i:(i+288)])/288)
    
d192[-1]

75.592135416666665

In [96]:
w12 = []
w24 = []
w48 = []
w72 = []
w96 = []
w192 = []
w288 = []
for i in range(len(spw)-287):
    w12.append(sum(spw.c[i:(i+12)])/12)
    w24.append(sum(spw.c[i:(i+24)])/24)
    w48.append(sum(spw.c[i:(i+48)])/48)
    w72.append(sum(spw.c[i:(i+72)])/72)
    w96.append(sum(spw.c[i:(i+96)])/96)
    w192.append(sum(spw.c[i:(i+192)])/192)
    w288.append(sum(spw.c[i:(i+288)])/288)
    
w192[-1]

98.623281249999991

## Quarterly Option Expirations Dates

In [122]:
exp = []
d = dp.parse("20/06/2015")
while d > dp.parse("20/03/1980"):
    d = d - WeekOfMonth(weekday = 4, week = 2)
    exp.append(d)
    d = d - DateOffset(days = 80)

len(exp)

142

In [123]:
w288[5]

1497.2060062152791

## Indicators

In [141]:
ltd = []
for i in range(len(spd)-287):
    t12 = ((spd.c[i]/(sum(spd.c[(i+6):(i+12)])/6)-1)*100)
    t24 = ((spd.c[i]/(sum(spd.c[(i+18):(i+24)])/6)-1)*100)
    t48 = ((spd.c[i]/(sum(spd.c[(i+42):(i+48)])/6)-1)*100)
    t72 = ((spd.c[i]/(sum(spd.c[(i+66):(i+72)])/6)-1)*100)
    t96 = ((spd.c[i]/(sum(spd.c[(i+90):(i+96)])/6)-1)*100)
    t192 = ((spd.c[i]/(sum(spd.c[(i+186):(i+192)])/6)-1)*100)
    t288 = ((spd.c[i]/(sum(spd.c[(i+282):(i+288)])/6)-1)*100)
    ltd.append((t12+t24+t48+t72+t96+t192+t288)/7)

In [142]:
ltw = []
for i in range(len(spw)-287):
    t12 = ((spw.c[i]/(sum(spw.c[(i+6):(i+12)])/6)-1)*100)
    t24 = ((spw.c[i]/(sum(spw.c[(i+18):(i+24)])/6)-1)*100)
    t48 = ((spw.c[i]/(sum(spw.c[(i+42):(i+48)])/6)-1)*100)
    t72 = ((spw.c[i]/(sum(spw.c[(i+66):(i+72)])/6)-1)*100)
    t96 = ((spw.c[i]/(sum(spw.c[(i+90):(i+96)])/6)-1)*100)
    t192 = ((spw.c[i]/(sum(spw.c[(i+186):(i+192)])/6)-1)*100)
    t288 = ((spw.c[i]/(sum(spw.c[(i+282):(i+288)])/6)-1)*100)
    ltw.append((t12+t24+t48+t72+t96+t192+t288)/7)

In [144]:
print ltd[0:20], ltw[0:20]

 [2.5236708189024477, 2.7336112845639065, 3.8704949283032062, 3.7400031223722849, 3.8215046018498215, 3.6183149677246882, 4.2860319305778036, 4.3397497714117002, 3.2455957631236636, 4.3368176761671213, 4.7276429486672544, 4.6265212388877917, 4.8187550529869929, 5.0645292951437284, 4.9256159784619342, 4.8733529826911024, 3.8096198320306036, 3.8344816040627241, 4.2032119104789247, 4.7319884198209587] [29.583573219566606, 31.080824117342868, 32.704590901031125, 32.667019785752402, 32.577663804584319, 32.551868639336028, 33.530277136673213, 31.758448338249927, 33.301940650769374, 31.902996304660643, 31.671760526377813, 34.352228860237112, 30.617815926876666, 31.476896641073861, 33.508602770550887, 33.885368888944022, 33.446182273808482, 31.432373174668992, 28.171551678283851, 32.300570202943284]
